<a href="https://colab.research.google.com/github/naty0611/Entregas/blob/main/Scraping_Computrabajo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# google-colab-selenium es un wrapper para facilitar usar Selenium en Google Colab
%pip install -q google-colab-selenium

# Importacion de librerias necesarias
import google_colab_selenium as gs  # librería que permite lanzar Chrome en Google Colab
from selenium.webdriver.common.by import By # localiza elementos por selectores (CSS, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait  # para aplicar esperas explícitas
from selenium.webdriver.support import expected_conditions as EC  # condiciones que usarán las esperas
import time, os, random  # librerías estándar para tiempos, carpetas y pausas aleatorias
import pandas as pd  # Crear y manejar DataFrames con los resultados
from google.colab import data_table   # Muestra tablas interactivas en Colab

In [3]:
# FUNCIONES AUXILIARES

def first_text(elem, selectors):
    """
    Devuelve el primer texto no vacío encontrado probando varios selectores CSS.
    - elem puede ser el driver o un WebElement.
    - selectors es una lista de strings CSS.
    """
    for sel in selectors:   # recorre cada selector de la lista
        try:
            found = elem.find_elements(By.CSS_SELECTOR, sel)  # busca elementos que coincidan
        except Exception:  # si falla el selector, no rompe el programa
            found = []
        for f in found:   # recorre cada elemento encontrado
            txt = f.text.strip()  # extrae y limpia el texto
            if txt:  # si no está vacío
                return txt  # lo devuelve inmediatamente el texto encontrado
    return ""  # si ninguno funcionó, devuelve string vacío


def find_requirements(driver):
    """
    Heurística para encontrar la lista de requisitos de la oferta.
    Busca en distintos lugares del HTML, probando en orden de prioridad:
    1. Lista <ul> con clase típica de requisitos
    2. Encabezado 'Requisitos' seguido de una lista
    3. Fallback: cualquier <ul> con >=2 <li> y palabras clave
    """
    # Caso típico: <ul> con clase específica
    try:
        lis = driver.find_elements(By.CSS_SELECTOR, "ul.fs16.disc.mbB li")  # busca los <li> dentro de ese <ul>
        if lis:   # si encontró algo
            return "; ".join([li.text.strip() for li in lis if li.text.strip()])  # une todo en un string
    except:
        pass

    # caso encabezado 'Requisitos' seguido de UL
    try:
        xpath = ("//h3[contains(translate(normalize-space(.),'REQUISITOS','requisitos'),'requisitos')]/"
                 "following-sibling::ul[1]/li | "
                 "//h4[contains(translate(normalize-space(.),'REQUISITOS','requisitos'),'requisitos')]/"
                 "following-sibling::ul[1]/li")
        lis = driver.find_elements(By.XPATH, xpath)  # busca los <li> justo después de un título con 'Requisitos'
        if lis:
            return "; ".join([li.text.strip() for li in lis if li.text.strip()])
    except:
        pass

    # Caso Fallback: cualquier lista <ul> con >=2 <li> y palabras clave
    try:
        uls = driver.find_elements(By.TAG_NAME, "ul")   # busca todas las listas <ul>
        for ul in uls:
            lis = ul.find_elements(By.TAG_NAME, "li")   # toma los <li> de esa lista
            if len(lis) >= 2:  # si hay al menos 2 elementos
                texts = [li.text.strip() for li in lis if li.text.strip()]  # extrae sus textos
                snippet = " ".join(texts[:3]).lower()  # junta los primeros 3 para analizarlos
                if any(k in snippet for k in ("experien", "conocim", "requisit", "competenc", "habilid")):
                    return "; ".join(texts)  # si hay palabras clave, devuelve la lista completa
    except:
        pass

    return ""

In [4]:
# CONFIGURACIÓN INICIAL

driver = gs.Chrome()   # inicializa navegador en Colab
base_url = "https://co.computrabajo.com/trabajo-de-analista-de-datos?p="  # URL base con pagina requerida
results = []           # lista donde se guardarán todas las ofertas
max_pages = 50         # número máximo de páginas a recorrer para obtener 1_000 ofertas
debug_folder = "/content/debug_html"   # carpeta donde se guardarán páginas con problemas
os.makedirs(debug_folder, exist_ok=True)  # crea la carpeta si no existe

<IPython.core.display.Javascript object>

In [5]:
# BUCLE PARA RECORRER LAS 50 PAGINAS QUE NECESITAMOS

for page in range(1, max_pages + 1):  # recorre páginas desde la 1 la 50+1
    print(f"Extrayendo página {page}...")  # muestra progreso de la extracción
    driver.get(base_url + str(page))  # abre la URL de la página

    # Espera a que carguen las tarjetas
    try:
        WebDriverWait(driver, 12).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "article.box_offer"))
        )   # espera máximo 12s a que aparezcan las ofertas
    except Exception:   # si no encuentra nada
        print(f"No se encontraron ofertas en la página {page}. Terminando scraping.")
        break  # Rompe el ciclo

    ofertas = driver.find_elements(By.CSS_SELECTOR, "article.box_offer")  # obtiene todas las tarjetas de la página
    if not ofertas:   # si la lista está vacía, termina el bucle
        break

    # Recorrer cada tarjeta de oferta
    for oferta_idx, oferta in enumerate(ofertas):   # recorre cada tarjeta de la página
        # Scroll para asegurar visibilidad
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", oferta)
        except:
            pass

        # Extraer título y link
        title = first_text(oferta, ["h2.fs18 a", "h2.fs18", "h2"])  # obtiene texto del título
        try:
            titulo_el = oferta.find_element(By.CSS_SELECTOR, "h2.fs18 a")  # busca el link del títul
            link = titulo_el.get_attribute("href")  # extraer el enlace del titulo
        except:
            try:
                a = oferta.find_element(By.TAG_NAME, "a")  # si no está en el <h2>, toma el primer <a>
                link = a.get_attribute("href")
            except:
                link = ""  # si no encontró nada, queda vacío

        # Empresa, ubicación y fecha
        company = first_text(oferta, ["a.fc_base.t_ellipsis", "div.company", "span.company"])
        location = first_text(oferta, ["span.mr10", "span.location"])
        date = first_text(oferta, ["p.fs13.fc_aux.mt15", "p.date"])

        # Datos de la tarjeta (salario y modalidad preliminar)
        detail_card_text = first_text(oferta, [
            "p.fs13.mt10 span.fc_base",
            "span.dIB.mr10:nth-child(1)",
            "p.fs13.mt10"
        ])

        salary_card = "No especificado"  # valor por defecto
        modality_card = ""   # valor por defecto

        # Detecta si el texto contiene salario
        if "$" in detail_card_text or "A convenir" in detail_card_text or "Mensual" in detail_card_text:
            salary_card = detail_card_text

        # Detecta modalidad si está en tarjeta
        if "Remoto" in detail_card_text or "Presencial" in detail_card_text or "Híbrido" in detail_card_text:
            if "Presencial" in detail_card_text: modality_card = "Presencial"
            elif "Remoto" in detail_card_text: modality_card = "Remoto"
            elif "Híbrido" in detail_card_text: modality_card = "Híbrido"

        # Inicializamos variables finales
        salary = salary_card
        description = ""
        requirements = ""
        modality = modality_card or "PENDIENTE_DE_CONFIRMACION"

        # Abrir detalle si existe link
        if link:
            original_handle = driver.current_window_handle  # guarda pestaña actual
            driver.execute_script("window.open('');")  # abre nueva pestaña vacía
            driver.switch_to.window(driver.window_handles[-1])  # cambia a la nueva pestaña
            driver.get(link)   # abre el link de la oferta

            # Espera a que cargue la página de detalle
            try:
                WebDriverWait(driver, 12).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.box_detail"))
                )
            except:
                WebDriverWait(driver, 8).until(
                    EC.presence_of_element_located((By.TAG_NAME, "body"))
                )

            # Extraer salario detallado
            salary_detail_text = first_text(driver, [
                "div.box_detail p.fs16.fc_base",
                "div.box_detail span.fc_base",
                "div.box_detail span.dIB.mr10",
                "p.salary", "span.salary"
            ])
            if "$" in salary_detail_text or "A convenir" in salary_detail_text:
                salary = salary_detail_text.split(" Contrato")[0].strip()

            # Extraer Descripción
            description = first_text(driver, [
                "div.box_detail > div.mbB",
                "div.box_detail div.mbB",
                "div.offer-description",
                "div[itemprop='description']",
                "div.description"
            ])

            # Extraer Requisitos
            requirements = find_requirements(driver)

            # Confirmar modalidad (si no estaba en tarjeta)
            if modality == "PENDIENTE_DE_CONFIRMACION":
                modality_detail_text = first_text(driver, [
                    "div.box_detail p.fs13 span.fc_base",
                    "div.box_detail span.dIB.mr10",
                ])
                clean_modality = ""
                if "Presencial" in modality_detail_text: clean_modality = "Presencial"
                elif "Remoto" in modality_detail_text: clean_modality = "Remoto"
                elif "Híbrido" in modality_detail_text: clean_modality = "Híbrido"
                modality = clean_modality or "Presencial"

            # Guardar HTML si faltan datos para depuración
            if not description or not requirements or salary in ("No especificado", ""):
                fname = os.path.join(debug_folder, f"debug_page_{page}_offer_{oferta_idx}.html")
                with open(fname, "w", encoding="utf-8") as f:
                    f.write(driver.page_source)
                print("Guardé HTML de depuración en:", fname)

            # Cerrar pestaña detalle y volver a la lista
            driver.close()
            driver.switch_to.window(original_handle)
        else:
            if modality not in ("Remoto", "Híbrido"): # Si no había link, asigna presencial por defecto salvo que diga remoto/híbrido
                modality = "Presencial"

        # Guardar la oferta en la lista de resultados
        results.append({
            "Titulo": title,
            "Empresa": company,
            "Ubicacion": location,
            "Fecha": date,
            "Modalidad": modality,
            "Salario": salary,
            "Requerimientos": requirements,
            "Descripcion": description,
            "Link": link
        })

    # Pausa aleatoria para evitar bloqueo
    time.sleep(random.uniform(1.0, 2.5))


# CERRAR DRIVER Y EXPORTAR

driver.quit()  # cierra el navegador
data_table.enable_dataframe_formatter()  # habilita visualización interactiva en Colab

df = pd.DataFrame(results)  # crea un DataFrame con los resultados
df.to_csv("computrabajo_ofertas_final.csv", index=False, encoding="utf-8-sig")  # exporta a CSV

print("Scraping completado. Registros extraídos:", len(df))   # imprime número de registros
df  # muestra el DF


Extrayendo página 1...
Guardé HTML de depuración en: /content/debug_html/debug_page_1_offer_3.html
Guardé HTML de depuración en: /content/debug_html/debug_page_1_offer_4.html
Guardé HTML de depuración en: /content/debug_html/debug_page_1_offer_7.html
Guardé HTML de depuración en: /content/debug_html/debug_page_1_offer_8.html
Guardé HTML de depuración en: /content/debug_html/debug_page_1_offer_9.html
Guardé HTML de depuración en: /content/debug_html/debug_page_1_offer_10.html
Guardé HTML de depuración en: /content/debug_html/debug_page_1_offer_13.html
Guardé HTML de depuración en: /content/debug_html/debug_page_1_offer_15.html
Guardé HTML de depuración en: /content/debug_html/debug_page_1_offer_18.html
Guardé HTML de depuración en: /content/debug_html/debug_page_1_offer_19.html
Extrayendo página 2...
Guardé HTML de depuración en: /content/debug_html/debug_page_2_offer_5.html
Guardé HTML de depuración en: /content/debug_html/debug_page_2_offer_14.html
Guardé HTML de depuración en: /conte

,Titulo,Empresa,Ubicacion,Fecha,Modalidad,Salario,Requerimientos,Descripcion,Link
0,Datamarshall,COVISIAN COLOMBIA S.A.S,"4,1",Hace 1 hora,Presencial,"$ 3.900.000,00 (Mensual)",Educación mínima: Universidad / Carrera Profes...,"$ 3.900.000,00 (Mensual) Contrato de Obra o la...",https://co.computrabajo.com/ofertas-de-trabajo...
1,"Analista de gestion conocimiento en nomina, fi...",,"Barranquilla, Atlántico",Hace 5 horas,Presencial,"$ 2.200.000,00 (Mensual)",Educación mínima: Universidad / Carrera Profes...,"$ 2.200.000,00 (Mensual) Contrato a término in...",https://co.computrabajo.com/ofertas-de-trabajo...
2,Analista de base de datos,TCC,"4,5",Hace 5 horas,Presencial,"$ 3.886.000,00 (Mensual)",Educación mínima: Universidad / Carrera Profes...,"$ 3.886.000,00 (Mensual) Contrato a término in...",https://co.computrabajo.com/ofertas-de-trabajo...
3,Analista de datos comerciales,RECORDAR PREVISION EXEQUIAL TOTAL S. A.S,"Bogotá, D.C., Bogotá, D.C.",Hace 5 horas,Presencial,No especificado,Educación mínima: Universidad / Carrera Profes...,A convenir Contrato a término indefinido Tiemp...,https://co.computrabajo.com/ofertas-de-trabajo...
4,Data Analyst,TALENTO,"Medellín, Antioquia",Hace 7 horas,Presencial,No especificado,Educación mínima: Universidad / Carrera técnic...,A convenir Contrato a término indefinido Tiemp...,https://co.computrabajo.com/ofertas-de-trabajo...
...,...,...,...,...,...,...,...,...,...
995,Asesor Comercial Bancario / Libranza,,"Medellín, Antioquia",Hace 2 días,Presencial,"$ 1.423.500,00 (Mensual) + Comisiones",Educación mínima: Universidad / Carrera técnic...,"$ 1.423.500,00 (Mensual) + Comisiones Contrato...",https://co.computrabajo.com/ofertas-de-trabajo...
996,Asesor/a Comercial,Agencia de Empleo de Comfenalco Antioquia,"Medellín, Antioquia",Hace 2 días,Presencial,"$ 2.000.000,00 (Mensual)",Educación mínima: Universidad / Carrera Profes...,"$ 2.000.000,00 (Mensual) Contrato a término fi...",https://co.computrabajo.com/ofertas-de-trabajo...
997,Consultor/a de belleza asesor/a perfumista,S&A Servicios y Asesorias,"4,5",Hace 2 días,Presencial,"$ 2.000.000,00 (Mensual)",Educación mínima: Bachillerato / Educación Med...,"$ 2.000.000,00 (Mensual) Contrato de Obra o la...",https://co.computrabajo.com/ofertas-de-trabajo...
998,"Asesor servicio al cliente, contratación estab...",ATENTO S.A.,"4,3",Hace 2 días,Presencial,No especificado,Educación mínima: Bachillerato / Educación Med...,A convenir Contrato a término indefinido Tiemp...,https://co.computrabajo.com/ofertas-de-trabajo...
